In [1]:
import time

def encode_open_accounts(data):
    #TODO --> Create a docstring
    #Number of open accounts: buckets (0-10, 11-20, mais de 20)
    data['Number.of.Open.Accounts.Buckets'] = data['Number.of.Open.Accounts']

    def encoding_noab(x):
        if x < 10:
            return '0-10'
        elif x > 20:
            return 'mais de 20'
        else:
            return '11-20'

    data['Number.of.Open.Accounts.Buckets'] = data['Number.of.Open.Accounts.Buckets'].apply(encoding_noab)
    data = data.drop(columns='Number.of.Open.Accounts').copy()
    return data

def credit_minus_loan(data):
    #TODO --> Create a docstring
    #Calculating current credit balance minus current loan amount
    #if 'Credit.Minus.Loan'> 0, then, 1 (esta pedindo emprestimo menor do que tem de saldo disponivel, entao, ok); otherwise, 0

    data['Credit.Minus.Loan'] = data['Current.Credit.Balance'] - data['Current.Loan.Amount']

    def encoding_cml(x):
        if x > 0:
            return 1
        else:
            return 0

    data['Credit.Minus.Loan'] = data['Credit.Minus.Loan'].apply(encoding_cml)
    return data

def encode_credit_problems(data):
    #TODO --> Create a docstring
    #Number of Number.of.Credit.Problems: 0, 1 ou mais de 1
    data['Number.of.Credit.Problems.Buckets'] = data['Number.of.Credit.Problems']

    def encoding_nocp(x):
        if x ==0:
            return 0
        elif x ==1:
            return 1
        else:
            return 'mais de 1'

    data['Number.of.Credit.Problems.Buckets'] = data['Number.of.Credit.Problems.Buckets'].apply(encoding_nocp)
    data = data.drop(columns='Number.of.Credit.Problems').copy()
    return data

def encode_years_job(data):
    #TODO --> Create a docstring
    data['Years.current_job_enc'] = data['Years.in.current.job'].map({'less than 1 year': 0,
                                                                             '1 year': 1,
                                                                             '2 years': 2,
                                                                             '3 years': 3,
                                                                             '4 years': 4,
                                                                             '5 years': 5,
                                                                             '6 years': 6,
                                                                             '7 years': 7,
                                                                             '8 years': 8,
                                                                             '9 years': 9,
                                                                             '10+ years': 10})
    data = data.drop(columns='Years.in.current.job').copy()
    return data

def encode_purpose(data):
    #TODO --> Create Purpose

    categories = ['other', 'moving', 'small_business', 'Take a Trip', 'major_purchase',
                   'wedding', 'Educational Expenses', 'vacation', 'renewable_energy']

    data['Purpose_enc'] = data['Purpose']

    for cat in categories:
        data.loc[data['Purpose_enc'] == cat, 'Purpose_enc'] = 'Other'

    data = data.drop(columns='Purpose').copy()
    return data

def encode_tax_liens(data):
    
    def encode_tl(x):
        if x == 0:
            x_enc = 0
        else:
            x_enc = 1
        return x_enc

    data['Tax.Liens.Enc'] = data['Tax.Liens'].map(encode_tl)
    data = data.drop(columns='Tax.Liens').copy()
    return data

def encode_leverage(data):
    
    data['leverage'] = (data['Monthly.Debt'] * 12) / data['Annual.Income']
    data = data.drop(columns='Monthly.Debt').copy()
    return data

def encode_credit_score(data):
    
    def encode_cs(x):
        if x >= 720:
            x_enc = 'Excellent'
        elif x >= 660:
            x_enc = 'Good'
        elif x >= 620:
            x_enc = 'Average'
        elif x >= 580:
            x_enc = 'Below Average'
        else:
            x_enc = 'Poor'
        return x_enc

    data['Credit.Score.Enc'] = data['Credit.Score'].map(encode_cs)
    data = data.drop(columns='Credit.Score').copy()
    return data

def encode_years_credit_history(data):
    
    data['Year.Credit.History.Enc'] = pd.cut(x=data['Years.of.Credit.History'],
                                  bins=[-1, 10, 20, 30, 100],
                                  labels=['0 a 10', '10.1 a 20', '20.1 a 30', '30+'])

    data = data.drop(columns='Years.of.Credit.History').copy()
    return data

def encode_barkruptcies(data):

    def encode_ban(x):
        if x == 0:
            x_enc = 0
        else:
            x_enc = 1
        return x_enc

    data['Bankruptcies.enc'] = data['Bankruptcies'].map(encode_ban)
    data = data.drop(columns='Bankruptcies').copy()
    return data

def encode_years_last_delinquent(data):
    
    data['Years.since.last.delinquent'] = data['Months.since.last.delinquent'] / 12
    
    data.loc[data['Years.since.last.delinquent'].isnull(), 'Years.since.last.delinquent'] = 100
    data['Years.since.last.delinquent'].isnull().sum()

    def encode_msld(x):
        if x <= 3:
            return 1
        else:
            return 0

    data['Years.since.last.delinquent'] = data['Years.since.last.delinquent'].map(encode_msld)
    data = data.drop(columns='Months.since.last.delinquent').copy()
    return data

def encode_loan_vs_income(data):
    # (12) Current loan amount / Annual income
    data['Loan.vs.Income'] = data['Current.Loan.Amount'] / data['Annual.Income']
    return data

def encode_total(data):

    data = encode_open_accounts(data)
    data = credit_minus_loan(data)
    data = encode_credit_problems(data)
    data = encode_years_job(data)
    data = encode_purpose(data)
    data = encode_tax_liens(data)
    data = encode_leverage(data)
    data = encode_credit_score(data)
    data = encode_years_credit_history(data)
    data = encode_barkruptcies(data)
    data = encode_years_last_delinquent(data)
    data = encode_loan_vs_income(data)
    data = data.drop(columns=['Loan.ID', 'Maximum.Open.Credit']).copy()
    
    return data


################
#  DECORATORS  #
################

def simple_time_tracker(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts))
        else:
            print(method.__name__, round(te - ts, 2))
        return result

    return timed

In [2]:
import pandas as pd
from google.cloud import storage

@simple_time_tracker
def get_data(nrows=None, local=True, **kwargs):
    """method to get the training data (or a portion of it) locally or from google cloud bucket"""
    # Add Client() here
    client = storage.Client()
    if local:
        path = "raw_data/train.csv" 
    else:
        # TODO
        # Create a google cloud storage path
        #path = "gs://{}/{}".format(BUCKET_NAME, BUCKET_TRAIN_DATA_PATH)
        pass

    data_original = pd.read_csv(path, nrows=nrows)
    
    return data_original

def clean_data(data_original):
    #TO-DO Create a docstring
    # (1) Deleting rows with Current.Loan.Amount equal to 99999999
    NUMBER_ERR = 99999999
    data = data_original[data_original['Current.Loan.Amount'] != NUMBER_ERR].copy()
    # (2) Deleting rows where Credit.Score and Annual.Income are NA
    data = data[data['Credit.Score'].notna()]
    data = data[data['Annual.Income'].notna()]
    # (3) Na coluna Home.Ownership, HaveMortgage substituido por Home Mortgage
    data['Home.Ownership'] = data['Home.Ownership'].replace(['HaveMortgage'], 'Home Mortgage')
    # (4) If Years.in.current.job is null, fill out with "less than 1 year"
    data['Years.in.current.job'] = data['Years.in.current.job'].fillna('less than 1 year')
    data['Years.in.current.job'] = data['Years.in.current.job'].replace(['less than  1 year'], 'less than 1 year')
    # (5) If Bankruptcies are null, substitute by 0
    data['Bankruptcies'] = data['Bankruptcies'].fillna(0)
    # (6) If Tax.Liens are null, substitute by 0
    data['Tax.Liens'] = data['Tax.Liens'].fillna(0)
    # (7) If Credit.Score > 850 (maximum), divide by 10 (assuming there was a typing error)
    #(e.g., instead of typing 750, person typed 7500) # Revisar pr numeros proximos a 850
    def div(x):
        if x > 850:
            return x / 10
        else:
            return x
    
    data['Credit.Score'] = data['Credit.Score'].apply(div)
    return data




In [3]:
params = dict(nrows=None,              
                  local=True)  # set to False to get data from GCP (Storage or BigQuery)

data_original = get_data(**params)
data = clean_data(data_original)
data_final = encode_total(data)

get_data 0.27


In [4]:
data_final

,Current.Loan.Amount,Term,Home.Ownership,Annual.Income,Current.Credit.Balance,Loan.Status,Number.of.Open.Accounts.Buckets,Credit.Minus.Loan,Number.of.Credit.Problems.Buckets,Years.current_job_enc,Purpose_enc,Tax.Liens.Enc,leverage,Credit.Score.Enc,Year.Credit.History.Enc,Bankruptcies.enc,Years.since.last.delinquent,Loan.vs.Income
0,17879,Short Term,Home Mortgage,95357.0,23986,1,mais de 20,1,0,6,Debt Consolidation,0,0.190000,Excellent,30+,0,1,0.187495
3,11200,Short Term,Rent,53965.0,11782,1,0-10,1,1,4,Debt Consolidation,0,0.301998,Excellent,10.1 a 20,1,0,0.207542
4,3608,Short Term,Home Mortgage,47709.0,22909,1,0-10,1,0,10,Home Improvements,0,0.085501,Excellent,10.1 a 20,0,0,0.075625
5,7877,Short Term,Home Mortgage,47707.0,1837,1,11-20,0,1,5,Home Improvements,0,0.127000,Excellent,10.1 a 20,1,0,0.165112
6,4732,Long Term,Rent,62721.0,0,0,0-10,0,0,10,Other,0,0.056798,Good,20.1 a 30,0,0,0.075445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60794,14885,Short Term,Rent,124875.0,14684,1,11-20,0,0,6,Debt Consolidation,0,0.047101,Excellent,0 a 10,0,0,0.119199
60795,5086,Short Term,Home Mortgage,50855.0,7671,1,11-20,1,0,1,Debt Consolidation,0,0.197000,Excellent,10.1 a 20,0,0,0.100010
60797,27840,Long Term,Home Mortgage,99430.0,34938,1,mais de 20,1,0,10,Debt Consolidation,0,0.292000,Below Average,30+,0,0,0.279996
60800,18229,Short Term,Rent,91143.0,8328,0,11-20,0,0,3,Debt Consolidation,0,0.103000,Excellent,10.1 a 20,0,1,0.200004


In [5]:
data_final.dtypes

Current.Loan.Amount                     int64
Term                                   object
Home.Ownership                         object
Annual.Income                         float64
Current.Credit.Balance                  int64
Loan.Status                             int64
Number.of.Open.Accounts.Buckets        object
Credit.Minus.Loan                       int64
Number.of.Credit.Problems.Buckets      object
Years.current_job_enc                   int64
Purpose_enc                            object
Tax.Liens.Enc                           int64
leverage                              float64
Credit.Score.Enc                       object
Year.Credit.History.Enc              category
Bankruptcies.enc                        int64
Years.since.last.delinquent             int64
Loan.vs.Income                        float64
dtype: object

In [6]:
import sys

In [7]:
sys.path
sys.path.append('/Users/sanderiwase/code/sanderiw/project493')
from wanna_buy_house import data
from wanna_buy_house import utils

In [8]:
data = data.final_treatment()

get_data 0.32


In [11]:
data

,Current.Loan.Amount,Term,Home.Ownership,Annual.Income,Current.Credit.Balance,Loan.Status,Number.of.Open.Accounts.Buckets,Credit.Minus.Loan,Number.of.Credit.Problems.Buckets,Years.current_job_enc,Purpose_enc,Tax.Liens.Enc,leverage,Credit.Score.Enc,Year.Credit.History.Enc,Bankruptcies.enc,Years.since.last.delinquent,Loan.vs.Income
0,17879,Short Term,Home Mortgage,95357.0,23986,1,mais de 20,1,0,6,Debt Consolidation,0,0.190000,Excellent,30+,0,1,0.187495
3,11200,Short Term,Rent,53965.0,11782,1,0-10,1,1,4,Debt Consolidation,0,0.301998,Excellent,10.1 a 20,1,0,0.207542
4,3608,Short Term,Home Mortgage,47709.0,22909,1,0-10,1,0,10,Home Improvements,0,0.085501,Excellent,10.1 a 20,0,0,0.075625
5,7877,Short Term,Home Mortgage,47707.0,1837,1,11-20,0,1,5,Home Improvements,0,0.127000,Excellent,10.1 a 20,1,0,0.165112
6,4732,Long Term,Rent,62721.0,0,0,0-10,0,0,10,Other,0,0.056798,Good,20.1 a 30,0,0,0.075445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60794,14885,Short Term,Rent,124875.0,14684,1,11-20,0,0,6,Debt Consolidation,0,0.047101,Excellent,0 a 10,0,0,0.119199
60795,5086,Short Term,Home Mortgage,50855.0,7671,1,11-20,1,0,1,Debt Consolidation,0,0.197000,Excellent,10.1 a 20,0,0,0.100010
60797,27840,Long Term,Home Mortgage,99430.0,34938,1,mais de 20,1,0,10,Debt Consolidation,0,0.292000,Below Average,30+,0,0,0.279996
60800,18229,Short Term,Rent,91143.0,8328,0,11-20,0,0,3,Debt Consolidation,0,0.103000,Excellent,10.1 a 20,0,1,0.200004


In [10]:
data.get_data()

AttributeError: 'DataFrame' object has no attribute 'get_data'